In [1]:
https://github.com/polyaxon/deep-learning-with-python-notebooks-on-polyaxon/blob/master/3.5-classifying-movie-reviews.ipynb

SyntaxError: invalid syntax (3708891057.py, line 1)

In [2]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


In [3]:
max([max(sequence) for sequence in train_data])


9999

In [4]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [5]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [6]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [7]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [8]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

C:\python37\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [11]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6968 - loss: 0.5977 - val_accuracy: 0.8554 - val_loss: 0.3992
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8993 - loss: 0.3253 - val_accuracy: 0.8855 - val_loss: 0.3050
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9240 - loss: 0.2347 - val_accuracy: 0.8681 - val_loss: 0.3213
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9353 - loss: 0.1926 - val_accuracy: 0.8882 - val_loss: 0.2787
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9531 - loss: 0.1497 - val_accuracy: 0.8854 - val_loss: 0.2915
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9599 - loss: 0.1266 - val_accuracy: 0.8665 - val_loss: 0.3480
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9644 - loss: 0.1130 - val_accuracy: 0.8773 - val_loss: 0.3137
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9738 - loss: 0.0931 - val_accuracy: 0.8833 - val

In [12]:
results = model.evaluate(x_test, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8561 - loss: 0.6444


In [13]:
results

[0.6338473558425903, 0.8578400015830994]

In [14]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras.layers import Bidirectional, LSTM

In [15]:
# 1. Load the IMDB dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [16]:
# 2. Pad or truncate the sequences to a fixed length of 250 words
x_train = pad_sequences(train_data, maxlen=250)
x_test = pad_sequences(test_data, maxlen=250)


In [17]:
# 3. Define a deep neural network architecture with embedding and bidirectional LSTM
model = models.Sequential()
model.add(layers.Embedding(10000, 128, input_length=250))  # Embedding layer for word embeddings
model.add(Bidirectional(LSTM(64)))  # Bidirectional LSTM layer
model.add(layers.Dense(16, activation='relu'))  # Fully connected layer
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification


C:\python37\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
# 4. Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
# 5. Train the model on the training set and validate on the validation set
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))


Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.5431 - loss: 0.6863 - val_accuracy: 0.7680 - val_loss: 0.5790
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7824 - loss: 0.5042 - val_accuracy: 0.8396 - val_loss: 0.3659
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.8794 - loss: 0.2965 - val_accuracy: 0.8638 - val_loss: 0.3192
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9285 - loss: 0.1931 - val_accuracy: 0.8771 - val_loss: 0.3260
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9577 - loss: 0.1295 - val_accuracy: 0.8654 - val_loss: 0.3425
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9644 - loss: 0.1121 - val_accuracy: 0.8700 - val_loss: 0.3886
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9784 - loss: 0.0758 - val_accuracy: 0.8656 - val_loss: 0.4040
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9882 - loss: 0.0481 - val_accuracy: 0.8637 - val_loss:

In [20]:
# 6. Evaluate the trained model on the test set and compute accuracy and loss
results = model.evaluate(x_test, test_labels)
print("Test loss:", results[0])
print("Test accuracy:", results[1])

782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - accuracy: 0.8407 - loss: 0.8971
Test loss: 0.8889256119728088
Test accuracy: 0.8416799902915955
